# Loading and plotting a fluorescence spectrum

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from agepy.spec.photons import Spectrum, DldAnodeUVW
from agepy import ageplot
ageplot.use("age")

Create an anode instance corresponding to the detector type and pass the rotation.

In [ ]:
anode = DldAnodeUVW(-98.8)

Load the data from the h5 file. Pass the correct h5 paths to the raw
data and if applicable the paths to recorded normalization data
(e.g. target density or upstream intensity or ...). 

In [ ]:
spectrum = Spectrum.from_h5(
    "spectrum.h5",  # Path to the h5 file
    anode,  # Anode object
    raw = "dld_rd#raw",  # Path to the raw data in the h5 file
    time = 600,  # Optional, acquisition time in seconds
    target_density = "baratron#value",  # Optional, path to the target density in the h5 file
    upstream_intensity = "mirror#value",  # Optional, path to the upstream intensity in the h5 file
)

Visualize the detector image:

In [ ]:
bins = np.histogram([], bins=256, range=(0, 1))[1]
fig, ax = spectrum.det_image(bins=bins, x_lim=(0.2, 0.8), y_lim=(0.3, 0.7))
plt.show()

Define a region of interest on the detector surface:

In [ ]:
# Define a ROI
roi = ((0.16, 0.80), (0.42, 0.52))

Create some dummy detector efficiencies with uncertainties that need to
be propagated:

In [ ]:
# Simulate detector efficiencies
qeff_x = np.linspace(0.1, 0.9, 30)
qeff_val = np.ones(30) * 0.9
qeff_err = np.ones(30) * 0.15

Create a dummy wavelength calibration with uncertainties that need to be
propagated:

In [ ]:
# Define wavelength calibration
a0 = (128.67, 0.07)
a1 = (38.94, 0.01)
calib = (a0, a1)

Choose a binning for the spectrum. For a calibrated spectrum the edges
need to have the same units as the calibration.

In [ ]:
# Define a binning for the spectrum
xedges = np.histogram([], bins=256, range=(a0[0], a0[0] + a1[0]))[1]

The errors can be propagated using Monte Carlo error propagation or
using a numerically calculated Jacobian matrix. The Jacobi method
underestimates the uncertainties, but the Monte Carlo method may take
a significantly longer time:

In [ ]:
#%%timeit
spec_jacobi, jacobi_errors = spectrum.spectrum(
    xedges,
    roi = roi,
    qeff = (qeff_val, qeff_err, qeff_x),
    calib = calib,
    uncertainties = "gauss",
)

In [ ]:
#%%timeit
spec_mc, mc_errors = spectrum.spectrum(
    xedges,
    roi = roi,
    qeff = (qeff_val, qeff_err, qeff_x),
    calib = calib,
    uncertainties = "montecarlo",
    mc_samples=10000
)

The resulting spectra are not identical, but similar enough within the
uncertainties. If the uncertainties of the efficiencies are large, the
Monte Carlo method will sometimes produce large outliers in the
uncertainties.

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].stairs(spec_jacobi, xedges, color=ageplot.colors[0], label="Jacobi")
ax[0].stairs(spec_jacobi + jacobi_errors, xedges, baseline=spec_jacobi - jacobi_errors, color=ageplot.colors[0], fill=True, alpha=0.5)
ax[1].stairs(spec_mc, xedges, color=ageplot.colors[1], label="Monte Carlo")
ax[1].stairs(spec_mc + mc_errors, xedges, baseline=spec_mc - mc_errors, color=ageplot.colors[1], fill=True, alpha=0.5)
ax[1].set_xlim(135, 160)
ax[1].set_xlabel("Wavelength [nm]")
ax[0].set_ylabel("Intensity [arb.u.]")
ax[1].set_ylabel("Intensity [arb.u.]")
ax[0].legend()
ax[1].legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.stairs(jacobi_errors, xedges, color=ageplot.colors[0], label="Jacobi")
ax.stairs(mc_errors, xedges, color=ageplot.colors[1], label="Monte Carlo")
ax.set_xlim(135, 160)
ax.set_xlabel("Wavelength [nm]")
ax.set_ylabel("Uncertainty [arb.u.]")
ax.legend()
plt.show()